In [10]:
import pandas as pd

df = pd.read_csv('file/health_result_label.csv')

# 질문 리스트
questions = df['question'].tolist()
answers = df['answer'].tolist()

df.head()

,question,answer,category02,label
0,"너무 목이 마른데요, 혹시 정수기 같은 건 없나요?",정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다.,대기실 및 진료실 위치 안내,0
1,교정기가 떨어진 것 같아서 붙이러 왔어요.,접수 먼저 부탁드립니다. 카운터에서 성함을 입력해 주시길 바랍니다.,진료 접수 안내,1
2,앞으로 얼마나 나아지는지 보고 아프면 오라고 하네요. 제가 다음에 예약하고 와도 되죠?,예약하실 때 지난 번 진료 기록이 있다고 말씀해 주시길 부탁 드립니다. 감사합니다.,다음 진료실 예약,2
3,이것도 수납용 키오스크인가요? 대기 줄이 너무 길어서요.,이곳에서도 수납 진행 가능합니다. 홈버튼을 누른 후 수납 버튼을 누르십시오.,수납 방법 안내,3
4,주사실 안에 가서 있으라는데 주사실이 어딘데요?,침을 맞는 곳이라 이름이 주사실입니다. 3층으로 올라가시면 됩니다.,대기실 및 진료실 위치 안내,0


In [7]:

df['label'].unique(), df['category02'].unique()

(array([0, 1, 2, 3, 4, 5, 6], dtype=int64),
 array(['대기실 및 진료실 위치 안내', '진료 접수 안내', '다음 진료실 예약', '수납 방법 안내', '약국 위치 문의',
        '증상 상담', '증빙 서류 발급'], dtype=object))

In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# 모델 구조를 정의한 코드
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=7)

# 모델 가중치를 CPU로 로드
model.load_state_dict(torch.load('model/health_model_weights.pth', map_location=torch.device('cpu'), weights_only=True))

tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import torch
import torch.nn.functional as F
import numpy as np

# numpy의 _reconstruct 함수를 허용 목록에 추가
torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

# 신뢰할 수 있는 데이터 파일을 로드
question_embeddings = torch.load('embeddings/train_question_embeddings.pth')

# 평가 모드로 설정
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def get_embedding(input_question, tokenizer, model):
    # 입력 문장을 토크나이즈
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        # hidden states를 포함하도록 설정
        outputs = model(**inputs, output_hidden_states=True)
        
        # 마지막 hidden state에서 [CLS] 토큰의 임베딩을 가져옴
        cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [CLS] 토큰의 임베딩
        
    return cls_embedding.squeeze().numpy()  # numpy 배열로 반환



# 코사인 유사도를 계산하여 가장 유사한 답변을 찾는 함수
def find_most_similar_answer_cosine(input_question, question_embeddings, answers, tokenizer, model):
    # 입력 질문 임베딩 생성
    input_embedding = get_embedding(input_question, tokenizer, model)

    max_similarity = -1
    best_answer = None
    
    # 각 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        # question_embedding을 텐서로 변환하고 차원 맞추기
        question_embedding_tensor = torch.tensor(question_embedding).unsqueeze(0)  # (1, 768)
        
        # input_embedding도 텐서로 변환하고 차원 맞추기
        input_embedding_tensor = torch.tensor(input_embedding).unsqueeze(0)  # (1, 768)
        
        # 코사인 유사도 계산
        similarity = F.cosine_similarity(input_embedding_tensor, question_embedding_tensor).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity  # 유사도 반환 추가

# 챗봇 응답 함수
def chatbot_response(input_question, tokenizer, model, question_embeddings, answers):
    # 1차 필터링: 분류 모델로 레이블 예측
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()

    # 2차 필터링: 같은 카테고리 내에서 코사인 유사도 계산
    # 같은 레이블의 질문들과 임베딩 필터링
    filtered_df = df[df['label'] == predicted_label]
    filtered_indices = filtered_df.index.tolist()

    # 필터링된 질문에 해당하는 미리 계산된 임베딩과 답변 가져오기
    filtered_question_embeddings = [question_embeddings[i] for i in filtered_indices]
    filtered_answers = [answers[i] for i in filtered_indices]

    # 코사인 유사도를 통해 가장 유사한 답변 찾기
    best_answer, cosine_similarity = find_most_similar_answer_cosine(input_question, filtered_question_embeddings, filtered_answers, tokenizer, model)
    
    return best_answer, cosine_similarity, predicted_label  # 세 가지 값 반환


C:\Users\tlsek\AppData\Local\Temp\ipykernel_15144\519318930.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load('embeddings/train_question_e

In [4]:
# 예시 질문
input_question = "너무 목이 마른데요, 혹시 정수기 같은 건 없나요?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)

예측된 레이블: 0
최고 유사도 답변: 정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다.
코사인 유사도: 1.0


In [21]:
# # 엑셀 파일에서 질문 데이터 읽어오기
# df_test = pd.read_excel('file/test_question.xlsx')
# test_questions = df_test.groupby('label')['question'].apply(list).to_dict()
# # 데이터프레임 생성
# questions_list = []

# for category, questions in test_questions.items():
#     for question in questions:
#         questions_list.append({'category': category, 'question': question})

# df_questions = pd.DataFrame(questions_list)

# df_questions['category'] = df_questions['category'].str.replace(r'^\d+\.', '', regex=True)

# # 새로운 라벨
# new_labels = {
#     '대기실 및 진료실 위치 안내': 0,
#     '진료 접수 안내': 1,
#     '다음 진료실 예약': 2,
#     '수납 방법 안내': 3,
#     '약국 위치 문의': 4,
#     '증상 상담': 5,
#     '증빙 서류 발급': 6
# }

# # 새로운 라벨링 적용
# df_questions['label'] = df_questions['category'].map(new_labels)

# # 데이터프레임 확인
# df_questions.head()
# df_questions.to_csv('result csv/test_questions_with_labels.csv', index=False)

,category,question,label
0,진료 접수 안내,화상연고도 처방이 되나요?,1
1,진료 접수 안내,오늘 두시 삼십분에 예약했어요. 언제 들어가나요?,1
2,진료 접수 안내,예약한 시간보다 조금 늦었어요. 다시 접수해야 하나요?,1
3,진료 접수 안내,"피가 계속 나는데, 저 죽는 걸까요?",1
4,진료 접수 안내,접수 재등록해도 되나요?,1


In [25]:
# test_questions 데이터프레임 읽기
test_questions = pd.read_csv('result csv/test_questions_with_labels.csv')

# 모든 질문을 리스트로 변환하여 input_questions에 저장
input_questions = test_questions['question'].tolist()
input_labels = test_questions['label'].tolist()  # 라벨 리스트 추가

# 결과를 저장할 리스트 초기화
results = []

# 각 질문에 대해 챗봇 응답 호출
for input_question, input_label in zip(input_questions, input_labels):
    # chatbot_response 함수 호출, 반환 값이 올바른지 확인
    best_answer, cosine_similarity, predicted_label = chatbot_response(
        input_question, tokenizer, model, question_embeddings, answers
    )
    
    # 결과를 리스트에 추가
    results.append({
        "question": input_question,
        "label": input_label,  # 원래 라벨 추가
        "predicted_label": predicted_label,  # 예측된 라벨 추가
        "best_answer": best_answer,
        "cosine_similarity": cosine_similarity
    })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과 출력
results_df[:30]


,question,label,predicted_label,best_answer,cosine_similarity
0,화상연고도 처방이 되나요?,1,0,진료실 옆 대기석에 등받이 있는 의자가 있습니다. 안내해 드리겠습니다.,0.602257
1,오늘 두시 삼십분에 예약했어요. 언제 들어가나요?,1,1,죄송합니다. 앞서 예약하신 손님이 있어서 예약시간대로 기다려주셔야 합니다.,0.993972
2,예약한 시간보다 조금 늦었어요. 다시 접수해야 하나요?,1,1,앞선 예약이 없어서 가능합니다. 예약시간 한 시간 미뤄 드리겠습니다.,0.973411
3,"피가 계속 나는데, 저 죽는 걸까요?",1,5,피부 사정에 따라 달라서 말씀드리기 어렵습니다. 증상은 사람마다 다릅니다.,0.994693
4,접수 재등록해도 되나요?,1,1,제가 안내해 드리겠습니다. 이쪽으로 따라오시면 됩니다.,0.989112
5,접수 시간이 지났는데 혹시 대기순번이 넘어갔나요?,1,1,확인을 도와드리겠습니다. 합의된 상황인지 확인이 필요합니다.,0.980679
6,접수가 왜이렇게 오래 걸려요?,1,1,앞에 보이는 접수증을 작성해 주시길 부탁드립니다.,0.988778
7,"다음에 들어가는 순번인데, 잠깐 화장실 다녀와도 되나요?",1,0,다녀오신 후 대기석에 앉아 잠시 대기해 주시기 바랍니다.,0.994216
8,접수하려고요.,1,1,현재 30분 정도 대기하셔야 합니다. 진료 접수 도와드리도록 하겠습니다.,0.969104
9,진료 접수 어떻게 하죠?,1,1,바로 도와드리겠습니다. 옆쪽 통로를 통해 접수창구로 가시면 됩니다.,0.994298


In [26]:
# 정확도 계산
correct_predictions = (results_df['label'] == results_df['predicted_label']).sum()  # 올바른 예측 수
total_predictions = results_df.shape[0]  # 총 예측 수
accuracy = correct_predictions / total_predictions  # 정확도 계산

# 정확도 출력
print(f'정확도: {accuracy * 100:.2f}%')

정확도: 90.14%


In [6]:
results_df.to_csv('result csv/chatbot_responses_trained.csv', index=False)